In [12]:
import autofit as af
import autolens as al
import autolens.plot as aplt

In [13]:
# grid
grid = al.Grid2D.uniform(
    shape_native=(50, 50),
    pixel_scales=0.001, 
)
    
# lens
BCG = al.Galaxy(redshift=0.338, mass=al.mp.NFW(centre=(0.00, 0.00), 
                                               ell_comps=(af.UniformPrior(lower_limit=0, upper_limit=1), af.UniformPrior(lower_limit=0, upper_limit=1)), 
                                               kappa_s= af.GaussianPrior(mean=0.5, sigma=0.3)) )
P1 = al.Galaxy(redshift=0.338, mass=al.mp.NFW(centre=(0.87, -16.42)))
P2 = al.Galaxy(redshift=0.338, mass=al.mp.NFW(centre=(-19.15, -13.37)))
P3 = al.Galaxy(redshift=0.338, mass=al.mp.NFW(centre=(5.12, 7.07)))

lens_sys = af.Collection(lens_0=BCG, lens_1=P1,lens_2=P2, lens_3=P3)

#sources - image plane - https://pyautolens.readthedocs.io/en/latest/overview/overview_8_point_sources.html

point_ds_H = al.PointDataset(
                name='H',
                positions=al.Grid2DIrregular([[-19.38, -1.74], [-8.33, -15.74], [2.37, -17.23], [6.37, 7.87]]),
                positions_noise_map=al.ArrayIrregular([0.05, 0.05, 0.05, 0.05]),
                )

point_ds_SN = al.PointDataset(
                name='SN',
                positions=al.Grid2DIrregular([[-1.93, -18.49], [-11.30, -15.32], [-18.80, -6.52]]),
                positions_noise_map=al.ArrayIrregular([0.05, 0.05, 0.05, 0.05]),
                )

# different redshift -> ignore for now

point_ds_3 = al.PointDataset(
                name='s3',
                positions=al.Grid2DIrregular([[4.22, 7.38], [-2.00, 8.60], [9.13, -3.22], [-3.41, -2.06]]),
                positions_noise_map=al.ArrayIrregular([0.05, 0.05, 0.05, 0.05]),
                )

point_dict = al.PointDict(point_dataset_list=[point_ds_H, point_ds_SN, point_ds_3])

print(point_dict)

#sources - source plane

#H = af.Model(al.ps.Point)

source_0 = af.Model(al.Galaxy, redshift=1.95, H=af.Model(al.ps.Point))

#SN = af.Model(al.ps.Point)

source_1 = af.Model(al.Galaxy, redshift=1.95, SN=af.Model(al.ps.Point))

#s3 = af.Model(al.ps.Point)

source_2 = af.Model(al.Galaxy, redshift=0.7663, s3=af.Model(al.ps.Point))

source_sys = af.Collection(source_0=source_0, source_1=source_1, source_2=source_2)

clusters = lens_sys + source_sys

model = af.Collection(galaxies=clusters)

print(model.info)

{'H': <autolens.point.point_dataset.PointDataset object at 0x0000025BD26348E0>, 'SN': <autolens.point.point_dataset.PointDataset object at 0x0000025BD7344F10>, 's3': <autolens.point.point_dataset.PointDataset object at 0x0000025BD2634730>}
Total Free Parameters = 7

model                                                                           Collection (N=7)
    galaxies                                                                    Collection (N=7)
        lens_0                                                                  Galaxy (N=0)
            mass                                                                NFW (N=0)
        lens_1                                                                  Galaxy (N=0)
            mass                                                                NFW (N=0)
        lens_2                                                                  Galaxy (N=0)
            mass                                                                

In [3]:
# fitting

solver = al.PointSolver(
    grid=grid,
    pixel_scale_precision=0.001,
    upscale_factor=2,
    distance_to_source_centre=0.01,
)

search = af.DynestyStatic(name="overview_point_source")

analysis = al.AnalysisPoint(point_dict=point_dict, solver=solver)

result = search.fit(model=model, analysis=analysis)

2023-09-08 00:50:39,609 - autofit.non_linear.abstract_search - INFO - Creating search
2023-09-08 00:50:39,612 - overview_point_source - INFO - Starting search
2023-09-08 00:50:39,616 - overview_point_source - INFO - Saving path info
2023-09-08 00:50:39,670 - overview_point_source - INFO - Not complete. Starting non-linear search.
2023-09-08 00:50:39,671 - overview_point_source - INFO - No Dynesty samples found, beginning new non-linear search. 
2023-09-08 00:50:39,736 - autofit.non_linear.initializer - INFO - Generating initial samples of model, which are subject to prior limits and other constraints.


KeyboardInterrupt: 